In [1]:
# extensions importing
import psycopg2

In [2]:
# constants for db connect
db_name = 'network_db'
pg_host = 'postgres_database'
sql_user = 'postgres'
sql_pwd = '12345'
port = '5432'
schema_name = 'public'

In [3]:
# database connicting
conn = psycopg2.connect(database=db_name,
                        host=pg_host,
                        user=sql_user,
                        password=sql_pwd,
                        port=port)

In [4]:
# database creating
with conn:
    with conn.cursor() as cursor:
        cursor.execute("""

            CREATE TYPE DATASET AS ENUM ('train', 'val', 'test');
        
            CREATE TABLE input_data_cell (
                file_id INTEGER NOT NULL CHECK (file_id > 0),
                row_id INTEGER NOT NULL CHECK (row_id > 0),
                detector_id INTEGER NOT NULL CHECK (detector_id > 0),
                time_values REAL[] NOT NULL,
                amplitude_values REAL[] NOT NULL,
                
                PRIMARY KEY(file_id, row_id, detector_id)
            );

            CREATE TABLE output_data_cell (
                file_id INTEGER NOT NULL CHECK (file_id > 0),
                row_id INTEGER NOT NULL CHECK (row_id > 0),
                detector_id INTEGER NOT NULL CHECK (detector_id > 0),
                defect_state BOOLEAN NOT NULL,
                defect_depth REAL NOT NULL,
                
                PRIMARY KEY(file_id, row_id, detector_id),
                
                FOREIGN KEY (file_id, row_id, detector_id) 
                    REFERENCES input_data_cell (file_id, row_id, detector_id)
            );

            CREATE TABLE dataset_crop_description (
                file_id INTEGER NOT NULL CHECK (file_id > 0),
                row_id INTEGER NOT NULL CHECK (row_id > 0),
                detector_id INTEGER NOT NULL CHECK (detector_id > 0),
                mirror_horizontal BOOLEAN NOT NULL,
                mirror_vertical  BOOLEAN NOT NULL,
                rotation_90_degree_count INTEGER NOT NULL CHECK (rotation_90_degree_count >= 0 AND rotation_90_degree_count < 4),
                crop_size INTEGER NOT NULL CHECK (crop_size > 0),
                dataset_name DATASET,
                
                PRIMARY KEY(file_id, 
                            row_id, 
                            detector_id,
                            mirror_horizontal,
                            mirror_vertical,
                            rotation_90_degree_count,
                            crop_size),
                            
                FOREIGN KEY (file_id, row_id, detector_id) 
                    REFERENCES input_data_cell (file_id, row_id, detector_id)
            );
            
            CREATE TABLE dataset_crop_prediction (
                file_id INTEGER NOT NULL CHECK (file_id > 0),
                row_id INTEGER NOT NULL CHECK (row_id > 0),
                detector_id INTEGER NOT NULL CHECK (detector_id > 0),
                mirror_horizontal BOOLEAN NOT NULL,
                mirror_vertical  BOOLEAN NOT NULL,
                rotation_90_degree_count INTEGER NOT NULL CHECK (rotation_90_degree_count >= 0 AND rotation_90_degree_count < 4),
                crop_size INTEGER NOT NULL CHECK (crop_size > 0),
                predict_session_num INTEGER NOT NULL CHECK (predict_session_num > 0),
                defect_state REAL NOT NULL,
                
                PRIMARY KEY (file_id, 
                            row_id, 
                            detector_id,
                            mirror_horizontal,
                            mirror_vertical,
                            rotation_90_degree_count,
                            crop_size,
                            predict_session_num),
                            
                FOREIGN KEY (file_id, 
                            row_id, 
                            detector_id,
                            mirror_horizontal,
                            mirror_vertical,
                            rotation_90_degree_count,
                            crop_size) REFERENCES 
                                dataset_crop_description (file_id, 
                                                        row_id, 
                                                        detector_id,
                                                        mirror_horizontal,
                                                        mirror_vertical,
                                                        rotation_90_degree_count,
                                                        crop_size)
            );
            
            """)
        
conn.commit()
conn.close()